In [1]:
import pickle as pkl

In [2]:
file = open('data/data_for_checking_loaders.pkl', 'rb')
data = pkl.load(file)
file.close()

/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
data

,formula,framework,composition,n_atoms_unrelaxed,n_atoms_opt,unrelaxed_cryst_id,unrelaxed_struct,opt_cryst_id,opt_struct,es_job_id,dft_energy,dft_energy_per_atom,dft_e_hull
0,Mn1O3Sb1,MnSbO3,"{'sites': {'A': ['Mn'], 'B': ['Sb'], 'X': ['O'...",5,40,137458332,[[7.27035675e-01 2.69595108e-19 0.00000000e+00...,151855989,"[[1.03668026 0. 0. ] Mn, [1.036...",51310512,-256.179186,-6.40448,0.413235
1,Cd1Ir1O3,IrCdO3,"{'sites': {'A': ['Ir'], 'B': ['Cd'], 'X': ['O'...",5,40,137458333,[[1.24166199e+00 1.75102687e-13 1.37403279e-14...,151918212,"[[3.21059535 1.93133496 1.93133496] Cd, [3.210...",51789972,-192.720815,-4.81802,0.993241
2,In1O3V1,VInO3,"{'sites': {'A': ['V'], 'B': ['In'], 'X': ['O']...",5,40,137458334,"[[1.10564595 0. 0. ] V, [2.2124...",151944346,"[[3.24304772 1.81490016 1.81490016] In, [3.243...",51895902,-249.674827,-6.241871,0.607629
3,Cu1O3Pb1,PbCuO3,"{'sites': {'A': ['Pb'], 'B': ['Cu'], 'X': ['O'...",5,40,137458335,[[ 2.15266114e-01 -2.17156472e-15 2.73644150e...,151882716,"[[2.25103269 1.88292668 1.88292668] Cu, [2.251...",51634093,-197.893852,-4.947346,0.340135
4,Hf1O3Ta1,TaHfO3,"{'sites': {'A': ['Ta'], 'B': ['Hf'], 'X': ['O'...",5,40,137458336,[[9.77294571e-01 2.24271159e-18 0.00000000e+00...,151860518,"[[1.94019034 2.11648954 2.11648954] Hf, [1.940...",51342231,-378.934765,-9.473369,0.792048
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3027,Be1Ce1O3,CeBeO3,"{'sites': {'A': ['Ce'], 'B': ['Be'], 'X': ['O'...",5,40,137465002,"[[2. 2. 2.] Be, [0. 0. 0.] Ce, [2. 2. 0.] O, [...",151928319,"[[1.80008086 1.80008086 1.80008086] Be, [1.800...",51853610,-306.891835,-7.672296,0.417001
4617,Ba7Co3La1Ni5O24,Ba0.875La0.125Co0.375Ni0.625O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Co', 'Ni'...",40,40,201883163,"[[8.000000e-06 8.000000e-06 4.000008e+00] Ba, ...",262017661,"[[0.02443428 0.02443428 4.06009044] Ba, [-0.01...",76676368,-225.50894,-5.637724,0.1881
7659,Ga1Na2Nb7O24Pb6,Na0.250Pb0.750Ga0.125Nb0.875O3,"{'sites': {'A': ['Na', 'Pb'], 'B': ['Ga', 'Nb'...",40,40,205585591,"[[8.e-06 8.e-06 8.e-06] Na, [2.000008 2.000008...",209624698,"[[1.97903265 2.03626107 2.03626107] Ga, [-0.21...",66018039,-296.983843,-7.424596,0.123989
1356,Ca1O3Pd1,CaPdO3,"{'sites': {'A': ['Ca'], 'B': ['Pd'], 'X': ['O'...",5,40,137459768,[[ 4.03700469e-01 -3.30211422e-18 -1.84657167e...,151883150,[[4.83670348e-01 4.58413030e-06 4.58413030e-06...,51654435,-227.154364,-5.678859,0.140539


In [4]:
from processing.utils import filter_data_by_properties,select_structures

In [5]:
data = filter_data_by_properties(data,"dft_e_hull")

In [6]:
data_unrelaxed = select_structures(data,"unrelaxed")

In [7]:
data_relaxed = select_structures(data,"relaxed")

In [8]:
from processing.interpolation.Interpolation import *

In [9]:
data_unrelaxed = apply_interpolation(data_unrelaxed,"dft_e_hull")

In [10]:
data_unrelaxed

,formula,framework,composition,n_atoms_unrelaxed,n_atoms_opt,unrelaxed_cryst_id,unrelaxed_struct,opt_cryst_id,opt_struct,es_job_id,dft_energy,dft_energy_per_atom,dft_e_hull,structure,dft_e_hull_interp,dft_e_hull_diff
6787,Ba1O24Rb7Ti7V1,Ba0.125Rb0.875Ti0.875V0.125O3,"{'sites': {'A': ['Ba', 'Rb'], 'B': ['V', 'Ti']...",40,40,205329467,"[[0. 0. 0.] Ba, [2. 2. 2.] V, [2. 2. 0.] O, [0...",213265840,"[[-0.04035941 -0.04035941 -0.04035941] Ba, [ 1...",69733340,-277.678013,-6.94195,0.234979,"[[-0.04035941 -0.04035941 -0.04035941] Ba, [ 1...",0.189777,0.045201
9726,La8Li4Mo4O24,LaLi0.500Mo0.500O3,"{'sites': {'A': ['La'], 'B': ['Li', 'Mo'], 'X'...",40,40,261839885,"[[3.90014939 7.84244649 6.30405396] La, [7.900...",261895880,"[[4.10394133 7.79910124 6.8948906 ] La, [8.021...",76546822,-300.865176,-7.521629,0.061616,"[[4.10394133 7.79910124 6.8948906 ] La, [8.021...",0.286327,-0.224711
7132,K4O24Pd6Rb4Ti2,K0.500Rb0.500Pd0.750Ti0.250O3,"{'sites': {'A': ['K', 'Rb'], 'B': ['Pd', 'Ti']...",40,40,205581929,"[[8.000000e-06 8.000000e-06 4.000008e+00] Rb, ...",252322577,[[1.25736912e-03 1.25736912e-03 1.02043710e-05...,72945225,-207.672065,-5.191802,0.204386,[[1.25736912e-03 1.25736912e-03 1.02043710e-05...,0.144915,0.059471
8412,Ba8O24Sc4Ti4,BaSc0.500Ti0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Sc', 'Ti'], 'X'...",40,40,205591139,"[[6.000008 2.000008 6.000008] Ba, [8.000000e-0...",209638031,"[[6.18453646 2.06151903 6.20163514] Ba, [6.184...",66019586,-308.160546,-7.704014,0.061506,"[[6.18453646 2.06151903 6.20163514] Ba, [6.184...",0.039036,0.022470
9712,Ba8Co4Mo4O24,BaCo0.500Mo0.500O3,"{'sites': {'A': ['Ba'], 'B': ['Co', 'Mo'], 'X'...",40,40,261839866,"[[6.000008 2.000008 6.000008] Ba, [6.000008 6....",262135472,"[[6.12076058 2.04026033 6.12076058] Ba, [6.120...",76676393,-266.930822,-6.673271,0.04587,"[[6.12076058 2.04026033 6.12076058] Ba, [6.120...",0.127175,-0.081304
6467,Ga2Na6O24Rb2Ru6,Na0.750Rb0.250Ga0.250Ru0.750O3,"{'sites': {'A': ['Rb', 'Na'], 'B': ['Ga', 'Ru'...",40,40,205328993,"[[0. 0. 0.] Rb, [2. 2. 2.] Ga, [2. 2. 0.] O, [...",209635943,"[[1.9306789 1.9306789 1.93017062] Ga, [1.930...",66019502,-236.523847,-5.913096,0.140415,"[[1.9306789 1.9306789 1.93017062] Ga, [1.930...",0.173263,-0.032848
6422,Ba3La5O24Rh4Sc4,Ba0.375La0.625Rh0.500Sc0.500O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Rh', 'Sc'...",40,40,205328922,"[[8.e-06 8.e-06 8.e-06] Ba, [2.000008 2.000008...",251413595,"[[ 0.08084181 -0.03145358 -0.03145358] Ba, [0....",72249684,-307.838545,-7.695964,0.047878,"[[ 0.08084181 -0.03145358 -0.03145358] Ba, [0....",0.146158,-0.098280
5619,Ba1La7O24Rh7Sc1,Ba0.125La0.875Rh0.875Sc0.125O3,"{'sites': {'A': ['Ba', 'La'], 'B': ['Sc', 'Rh'...",40,40,205327668,"[[0. 0. 0.] Ba, [2. 2. 2.] Sc, [2. 2. 0.] O, [...",252539831,"[[0.04235672 0.04235672 0.04235672] Ba, [0.235...",72945561,-296.665427,-7.416636,0.155793,"[[0.04235672 0.04235672 0.04235672] Ba, [0.235...",0.198375,-0.042582
6666,Ga2Na1O24Rb7Ru6,Na0.125Rb0.875Ga0.250Ru0.750O3,"{'sites': {'A': ['Na', 'Rb'], 'B': ['Ga', 'Ru'...",40,40,205329293,"[[8.e-06 8.e-06 8.e-06] Na, [2.000008 2.000008...",208170001,"[[1.96467545 1.98130837 1.98130837] Ga, [1.964...",65084855,-232.65037,-5.816259,0.21097,"[[1.96467545 1.98130837 1.98130837] Ga, [1.964...",0.238211,-0.027241
9521,Ca8Fe4Ni4O24,CaFe0.500Ni0.500O3,"{'sites': {'A': ['Ca'], 'B': ['Fe', 'Ni'], 'X'...",40,40,222998288,"[[6.000008 2.000008 2.000008] Ca, [8.000000e-0...",253622152,"[[5.75036498 1.93569576 1.93569576] Ca, [5.750...",73354879,-231.231449,-5.780786,0.149435,"[[5.75036498 1.93569576 1.93569576] Ca, [5.750...",0.149399,0.000036


In [11]:
from processing.dataloader.dataloader import get_dataloaders
from processing.create_model.create_model import create_model
from training.hyperparameters.default import get_default_cgcnn_hyperparameters, get_default_painn_hyperparameters, get_default_e3nn_hyperparameters
from training.model_training.trainer import trainer

In [12]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","CGCNN",4)
model, Normalizer = create_model("CGCNN",train_loader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 1172.85it/s]


In [13]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","CGCNN",4)
model, Normalizer = create_model("CGCNN",train_loader)
hyperparameters = get_default_cgcnn_hyperparameters()
print(hyperparameters)
best_model, loss_fn = trainer(model,Normalizer,'CGCNN',train_loader,val_loader,hyperparameters,"saved_models/CGCNN/Scrap",0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18/18 [00:00<00:00, 2745.66it/s]


{'MaxEpochs': 100, 'log_lr': -3, 'reduceLR_patience': 10, 'atom_fea_len': 64, 'n_conv': 3, 'h_fea_len': 128, 'n_h': 1}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  4.87it/s]


tensor([1.1213, 1.1569, 1.1144, 1.2914], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([-1.2717e-01, -2.7241e-02, -8.1304e-02,  7.6190e-05], device='cuda:0')
tensor([0.7084, 0.7139, 0.5512, 0.6853], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([-2.2471e-01,  5.9471e-02,  3.6088e-05, -9.8280e-02], device='cuda:0')
tensor([0.4289, 0.1165, 0.5078], device='cuda:0', grad_fn=<ViewBackward0>)
tensor([-0.0038, -0.0426, -0.0129], device='cuda:0')
   1 ,lr = 0.00100000  train loss = 0.06828405  val loss = 0.09824877  time = 00:00:00


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 145.21it/s]


...

tensor([-1.9986,  0.8411,  0.7336, -0.0799], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([-2.2471e-01,  7.6190e-05, -1.2877e-02, -8.1304e-02], device='cuda:0')
tensor([ 1.0514,  0.2120, -0.7721], device='cuda:0', grad_fn=<ViewBackward0>)
tensor([ 5.9471e-02,  3.6088e-05, -4.2582e-02], device='cuda:0')
  99 ,lr = 0.00000391  train loss = 0.02277896  val loss = 0.09432002  time = 00:00:00



  0%|                                                                                                                                                                                      | 0/3 [00:00<?, ?it/s]

tensor([ 0.0104,  1.2665, -1.0863,  0.2379], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([-0.0272,  0.0595, -0.1272, -0.0129], device='cuda:0')
tensor([ 0.7526, -2.1911,  0.6890,  0.3579], device='cuda:0',
       grad_fn=<ViewBackward0>)
tensor([-3.8482e-03, -2.2471e-01,  7.6190e-05, -4.2582e-02], device='cuda:0')
tensor([ 0.6967, -0.3288, -0.4192], device='cuda:0', grad_fn=<ViewBackward0>)
tensor([ 3.6088e-05, -8.1304e-02, -9.8280e-02], device='cuda:0')


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 96.07it/s]


 100 ,lr = 0.00000391  train loss = 0.02222338  val loss = 0.09036792  time = 00:00:00


In [14]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","Painn",4)
model, Normalizer = create_model("Painn",train_loader)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 5777.28it/s]


In [15]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","Painn",4)
model, Normalizer = create_model("Painn",train_loader)
hyperparameters = get_default_painn_hyperparameters()
best_model, loss_fn = trainer(model,Normalizer,'Painn',train_loader,val_loader,hyperparameters,"saved_models/Painn/Scrap",0)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 3514.78it/s]


 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_dft_e_hull_diff | GPU Memory (MB)


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 2/3 [00:00<00:00, 29.88it/s]

39:04 |     1 |     1.000e-03 |     1.5171 |          0.2393 |              0.2393 |             111



 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 2/3 [00:00<00:00, 25.62it/s]

...

39:14 |    98 |     7.813e-06 |     0.0042 |          0.0543 |              0.0543 |             127


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 2/3 [00:00<00:00, 25.11it/s]


39:14 |    99 |     7.813e-06 |     0.0037 |          0.0549 |              0.0549 |             127


 67%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████                                                          | 2/3 [00:00<00:00, 24.87it/s]

39:15 |   100 |     7.813e-06 |     0.0035 |          0.0539 |              0.0539 |             127


In [16]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","e3nn",4)
model, Normalizer = create_model("e3nn",train_loader)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 27.10it/s]
/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


In [17]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","e3nn",4)
model, Normalizer = create_model("e3nn",train_loader)
hyperparameters = get_default_e3nn_hyperparameters()
print(hyperparameters)
best_model, loss_fn = trainer(model,Normalizer,'e3nn',train_loader,val_loader,hyperparameters,"saved_models/e3nn/Scrap",0)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 27.18it/s]
/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/jit/_check.py:181: UserWarning: The TorchScript type system doesn't support instance-level annotations on empty non-base types in `__init__`. Instead, either 1) use a type annotation in the class body, or 2) wrap the type in `torch.jit.Attribute`.
  warnings.warn("The TorchScript type system doesn't support "


{'MaxEpochs': 100, 'log_lr': -3, 'reduceLR_patience': 10, 'len_embedding_feature_vector': 64, 'num_hidden_feature': 128, 'num_hidden_layer': 2, 'multiplicity_irreps': 32, 'num_conv': 3, 'num_radical_basis': 10, 'num_radial_neurons': 100}


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.42it/s]


   1 ,lr = 0.00100000  train loss = 0.05843474  val loss = 0.05371357  time = 00:00:02


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 13.26it/s]


...

  98 ,lr = 0.00001563  train loss = 0.02978511  val loss = 0.06001214  time = 00:00:00


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14.07it/s]


  99 ,lr = 0.00001563  train loss = 0.02976468  val loss = 0.06003237  time = 00:00:00


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 14.42it/s]


 100 ,lr = 0.00000781  train loss = 0.02974012  val loss = 0.06003180  time = 00:00:00


In [18]:
train_loader, val_loader, test_loader = get_dataloaders(data_unrelaxed,"dft_e_hull","e3nn_contrastive",4)
model, Normalizer = create_model("e3nn",train_loader)

100%|██████████| 18/18 [00:00<00:00, 2842.74it/s]


In [14]:
import torch
loss_fn = torch.nn.L1Loss()

In [15]:
A = torch.tensor([1.0,2.0,3.0])

In [16]:
B = torch.tensor([[1.0],[2.0],[3.0]])

In [17]:
loss_fn(A,B)

/home/jkdamewo/miniconda3/envs/Perovskite_ML_Environment/lib/python3.10/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([3, 1])) that is different to the input size (torch.Size([3])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


tensor(0.8889)